In [ ]:
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import requests
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
chrome_driver = ChromeDriverManager().install() # 크롬 드라이버 버전 변경 시 자동 설치

# chrome_driver = ChromeDriverManager(version="114.0.5735.90").install()

# 페이지 선택해서 크롤링하는 함수(이미지 포함)

In [ ]:
def crawling_melon_pg(search, num): # 오류 해결한 버전
    label = input("label을 입력하세요: ")
    
    options = Options()
    options.add_argument('headless')
    options.add_argument('--start-maximized')
    options.add_experimental_option('detach', True)
    driver = webdriver.Chrome(options=options)
    #Selenium_driver로 url 접속
    url = "https://www.melon.com/dj/themegenre/djthemegenre_list.htm" # 멜론 테마 장르 url
    driver.get(url)
    #Selenium_접속하는 데 시간이 걸릴 수 있으므로 1초 기다림
    time.sleep(1)
    #Selenium x_path
    x_path = '//*[@id="djSearchKeyword"]' # 태그 검색 박스
    searchbox = driver.find_element_by_xpath(x_path)
    #버튼 클릭
    searchbox.click()
    #키워드 입력
    searchbox.send_keys(search)
    #버튼 클릭
    searchbox.send_keys(Keys.ENTER)


    title_list = []
    artitst_list = []
    likes_list = []
    album_image_link_list = []
    tag_list = []

    title_error_list = []
    artist_error_list = []

    for i in tqdm(range(num)):
        elements = driver.find_elements_by_class_name('thumb')
        for idx in range(len(elements)):
            es = driver.find_elements_by_class_name('thumb')
            time.sleep(1)
            es[idx].click()
            # Selenium_위의 작업이 수행된 후 해당 접속 사이트 url 정보 가져옴(html)
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            titles_elem = soup.select('tbody > tr > td > div.wrap > div.wrap_song_info > div.ellipsis.rank01 > span > a')
            artists_elem = soup.select('tbody > tr > td > div.wrap > div.wrap_song_info > div.ellipsis.rank02 > span')
            likes_elem = soup.select('tbody > tr > td > div.wrap > button > span.cnt')
            tag_elem = soup.select('div.entry > div.tag_list.type03 > a > span')
            album_elem = soup.select('tbody > tr > td:nth-child(3) > div > a ')

            tag_tmp = []
            for tag in tag_elem:
                tag_tmp.append(tag.text.split('#')[-1])
            for likes in likes_elem:
                likes_list.append(likes.text.split('\n')[-1])
            for title, artist, album in zip(titles_elem, artists_elem, album_elem):
                if '/' in title.text:
                    modified_title = title.text.replace('/', '%2F')
                    title_error_list.append(title.text)
                elif '"' in title.text:
                    modified_title = title.text.replace('"', '%2F')
                    title_error_list.append(title.text)
                elif '?' in title.text:
                    modified_title = title.text.replace('?', '%2F')
                    title_error_list.append(title.text)
                elif '*' in title.text:
                    modified_title = title.text.replace('*', '%2F')
                    title_error_list.append(title.text)
                elif '<' in title.text:
                    modified_title = title.text.replace('<', '%2F')
                    title_error_list.append(title.text)
                elif '>' in title.text:
                    modified_title = title.text.replace('>', '%2F')
                    title_error_list.append(title.text)
                elif ':' in title.text:
                    modified_title = title.text.replace(':', '%2F')
                    title_error_list.append(title.text)
                else:
                    modified_title = title.text

                if '/' in artist.text:
                    modified_artist = artist.text.replace('/', '%2F')
                    artist_error_list.append(artist.text)
                elif '"' in artist.text:
                    modified_artist = artist.text.replace('"', '%2F')
                    artist_error_list.append(artist.text)
                elif '?' in artist.text:
                    modified_artist = artist.text.replace('?', '%2F')
                    artist_error_list.append(artist.text)
                elif '*' in artist.text:
                    modified_artist = artist.text.replace('*', '%2F')
                    artist_error_list.append(artist.text)
                else:
                    modified_artist = artist.text

                title_list.append(modified_title)
                tag_list.append(tag_tmp)
                artitst_list.append(artist.text)

                imgUrl = album.find('img')['src']
                album_image_link_list.append(imgUrl)
                #urlretrieve는 다운로드 함수
                urllib.request.urlretrieve(imgUrl, '../image/'+label+'/'+search+'/'+modified_title+'_'+modified_artist+'.jpg')

            driver.back()
        time.sleep(1)
        page_elem = driver.find_elements_by_class_name('paging > span > a')
        try:
            page_elem[i].click()
            time.sleep(2)
        except IndexError:
            pass
            time.sleep(1)

    driver.close()
    search_list = [search]*len(title_list)

    df = pd.DataFrame({'title': title_list, 'artist': artitst_list, 'likes': likes_list,
                       'tag': search_list, 'specific_tag': tag_list, 'image': album_image_link_list})
    df['likes'] = df['likes'].str.replace(',', '').astype(int) # object타입을 int타입으로 변경
    return df, title_error_list, artist_error_list

# 데이터프레임 저장

In [ ]:
#중복 행 지우기
df_rain = df_rain.drop_duplicates(keep='first',subset='title', ignore_index=True)
print("중복 제거 후 행 개수: ",len(df_rain))

In [ ]:
#데이터 프레임 저장
df_rain.to_csv('../output/{}.csv'.format('비오는날'),encoding='utf-8-sig',index=False)

In [ ]:
# 데이터 프레임 불러오기
df = pd.read_csv('../output/비오는날.csv')
df

### sunny

In [ ]:
df_sunny_1, title_error_sunny_1, artist_error_sunny_1 = crawling_melon_pg('맑은날', 3)

In [ ]:
#데이터 프레임 저장
df_sunny_1.to_csv('../output/{}.csv'.format('맑은날'),encoding='utf-8-sig',index=False)

In [ ]:
df_sunny_2, title_error_sunny_2, artist_error_sunny_2 = crawling_melon_pg('화창한날', 3)

In [ ]:
#데이터 프레임 저장
df_sunny_2.to_csv('../output/{}.csv'.format('화창한날'),encoding='utf-8-sig',index=False)

In [ ]:
df_sunny_3, title_error_sunny_3, artist_error_sunny_3 = crawling_melon_pg('여행', 4)

In [ ]:
#데이터 프레임 저장
df_sunny_3.to_csv('../output/{}.csv'.format('여행'),encoding='utf-8-sig',index=False)

In [ ]:
title_error_sunny = title_error_sunny_1 + title_error_sunny_2 + title_error_sunny_3
artist_error_sunny = artist_error_sunny_1 + artist_error_sunny_2 + artist_error_sunny_3

### rainy

In [ ]:
df_rainy_1, title_error_rainy_1, artist_error_rainy_1 = crawling_melon_pg('비오는날', 4)

In [ ]:
#데이터 프레임 저장
df_rainy_1.to_csv('../output/{}.csv'.format('비오는날'),encoding='utf-8-sig',index=False)

In [ ]:
df_rainy_2, title_error_rainy_2, artist_error_rainy_2 = crawling_melon_pg('장마', 4)

In [ ]:
#데이터 프레임 저장
df_rainy_2.to_csv('../output/{}.csv'.format('장마'),encoding='utf-8-sig',index=False)

In [ ]:
df_rainy_3, title_error_rainy_3, artist_error_rainy_3 = crawling_melon_pg('여름비', 2)

In [ ]:
#데이터 프레임 저장
df_rainy_3.to_csv('../output/{}.csv'.format('여름비'),encoding='utf-8-sig',index=False)`

In [ ]:
title_error_rainy = title_error_rainy_1 + title_error_rainy_2 + title_error_rainy_3
artist_error_rainy = artist_error_rainy_1 + artist_error_rainy_2 + artist_error_rainy_3

### foggy

In [ ]:
df_foggy_1, title_error_foggy_1, artist_error_foggy_1 = crawling_melon_pg('안개', 1)

In [ ]:
#데이터 프레임 저장
df_foggy_1.to_csv('../output/{}.csv'.format('안개'),encoding='utf-8-sig',index=False)

In [ ]:
df_foggy_2, title_error_foggy_2, artist_error_foggy_2 = crawling_melon_pg('미세먼지', 5)

In [ ]:
#데이터 프레임 저장
df_foggy_2.to_csv('../output/{}.csv'.format('미세먼지'),encoding='utf-8-sig',index=False)

In [ ]:
df_foggy_3, title_error_foggy_3, artist_error_foggy_3 = crawling_melon_pg('흐린날', 3)

In [ ]:
#데이터 프레임 저장
df_foggy_3.to_csv('../output/{}.csv'.format('흐린날'),encoding='utf-8-sig',index=False)

In [ ]:
title_error_foggy = title_error_foggy_1 + title_error_foggy_2 + title_error_foggy_3
artist_error_foggy = artist_error_foggy_1 + artist_error_foggy_2 + artist_error_foggy_3

### snowy

In [ ]:
df_snowy_1, title_error_snowy_1, artist_error_snowy_1 = crawling_melon_pg('눈오는날', 4)

In [ ]:
df_snowy_2, title_error_snowy_2, artist_error_snowy_2 = crawling_melon_pg('크리스마스', 4)

In [ ]:
#데이터 프레임 저장
df_snowy_1.to_csv('../output/{}.csv'.format('눈오는날'),encoding='utf-8-sig',index=False)

In [ ]:
#데이터 프레임 저장
df_snowy_2.to_csv('../output/{}.csv'.format('크리스마스'),encoding='utf-8-sig',index=False)

### sunrise_sunset

In [ ]:
df_sunrise_sunset_1 = crawling_melon_pg('일출', 1)

In [ ]:
df_sunrise_sunset_2 = crawling_melon_pg('일몰', 1)

In [ ]:
df_sunrise_sunset_3 = crawling_melon_pg('저녁노을', 1)

In [ ]:
df_sunrise_sunset_4 = crawling_melon_pg('노을', 6)

In [ ]:
#데이터 프레임 저장
df_sunrise_sunset_1.to_csv('../output/{}.csv'.format('일출'),encoding='utf-8-sig',index=False)

In [ ]:
#데이터 프레임 저장
df_sunrise_sunset_2.to_csv('../output/{}.csv'.format('일몰'),encoding='utf-8-sig',index=False)

In [ ]:
#데이터 프레임 저장
df_sunrise_sunset_3.to_csv('../output/{}.csv'.format('저녁노을'),encoding='utf-8-sig',index=False)

In [ ]:
#데이터 프레임 저장
df_sunrise_sunset_4.to_csv('../output/{}.csv'.format('노을'),encoding='utf-8-sig',index=False)

In [ ]:
# 데이터 프레임 불러오기
df_맑은날 = pd.read_csv('../output/sunny/맑은날.csv')
df_여행 = pd.read_csv('../output/sunny/여행.csv')
df_화창한날 = pd.read_csv('../output/sunny/화창한날.csv')

df_미세먼지 = pd.read_csv('../output/foggy/미세먼지.csv')
df_안개 = pd.read_csv('../output/foggy/안개.csv')
df_흐린날 = pd.read_csv('../output/foggy/흐린날.csv')

df_눈오는날 = pd.read_csv('../output/snowy/눈오는날.csv')
df_크리스마스 = pd.read_csv('../output/snowy/크리스마스.csv')

df_비오는날 = pd.read_csv('../output/rainy/비오는날.csv')
df_여름비 = pd.read_csv('../output/rainy/여름비.csv')
df_장마 = pd.read_csv('../output/rainy/장마.csv')

df_일몰 = pd.read_csv('../output/sunrise_sunset/일몰.csv')
df_일출 = pd.read_csv('../output/sunrise_sunset/일출.csv')
df_저녁노을 = pd.read_csv('../output/sunrise_sunset/저녁노을.csv')
df_노을 = pd.read_csv('../output/sunrise_sunset/노을.csv')

In [ ]:
print(len(df_맑은날), len(df_여행), len(df_화창한날))

In [ ]:
print(len(df_미세먼지), len(df_안개), len(df_흐린날))

In [ ]:
print(len(df_눈오는날), len(df_크리스마스))

In [ ]:
print(len(df_비오는날), len(df_여름비), len(df_장마))

In [ ]:
print(len(df_일몰), len(df_일출), len(df_저녁노을), len(df_노을))